<a href="https://colab.research.google.com/github/hkhong72/big_data/blob/main/221020_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Maven Clean , Maven install

home 디렉터리 밑에 
work밑에 
mkdir java


c아래 javaworkspace 아래 hadoopmr 아래 target 아래 jar파일생성확인 및 경로복사

윈도우 터미널에서 cd C:\java_workspace\hadoopMR\target (경로)

ls로 hadoopMR-0.1.jar 확인


 scp .\hadoopMR-0.1.jar ubuntu:~/work/java 

 윈도우에서 리눅스로 보내기.

start-dfs.sh

start-yarn.sh

jps

mapred historyserver start&

exit

cd work

cd java

yarn jar ./hadoopMR-0.1.jar com.adacho.driver.WordCount sample-input sample-out
put2

hdfs dfs -ls sample-output2 로 확인

hdfs dfs -cat sample-output2/part-r-00000 

만약 우리가올린파일이 128MB넘으면 00001 ~ 해서 파티션이 생김

work 아래에 

mkdir air-data

AirOnTimeCSV.zip 슬랙에서 다운 후 압축풀기

cd 원하는파일경로

scp .\airOT2012*.csv ubuntu:~/work/air-data/

리눅스에서 
work/air-data에서
모든 csv
vi .csv
dd로 하나씩 헤더부분지우고저장

hdfs dfs -mkdir air-input 에어인풋만들기

hdfs dfs -put ./* air-input 현재위치에 csv모든것을 air-input 으로 보낸다

새로운 maven project

com.adacho

airPerformance

0.1 

jar

pom.xml그대로 가져오기

com.adacho.common 패키지생성

AirlinePerformanceParser 클래스생성


In [ ]:
package com.adacho.common;

import org.apache.hadoop.io.Text;

public class AirlinePerformanceParser {
	private int year;
	private int month;
	private int day;
	
	private int arriveDelayTime = 0;
	private int departureDelayTime = 0;
	private int distance = 0;
	
	private boolean arriveDelayAvailable = true;
	private boolean departureDelayAvailable = true;
	private boolean distanceAvailable = true;
	
	private String uniqueCarrier;
	
	public AirlinePerformanceParser(Text text) {
		try {
			String[] columns = text.toString().split(",");
			
			year = Integer.parseInt(columns[0]);
			month = Integer.parseInt(columns[1]);
			day = Integer.parseInt(columns[2]);
			uniqueCarrier = columns[5];
			
			if(!columns[16].equals("")) {
				departureDelayTime = (int)Float.parseFloat(columns[16]);
			}else {
				departureDelayAvailable = false;
			}
			
			if(!columns[26].equals("")) {
				arriveDelayTime = (int)Float.parseFloat(columns[26]);
			}else {
				arriveDelayAvailable = false;
			}
			
			if(!columns[37].equals("")) {
				distance = (int)Float.parseFloat(columns[37]);
			}else {
				distanceAvailable = false;
			}
		} catch (Exception e) {
			
		}
	}

	public int getYear() {
		return year;
	}

	public int getMonth() {
		return month;
	}

	public int getDay() {
		return day;
	}

	public int getArriveDelayTime() {
		return arriveDelayTime;
	}

	public int getDepartureDelayTime() {
		return departureDelayTime;
	}

	public int getDistance() {
		return distance;
	}

	public boolean isArriveDelayAvailable() {
		return arriveDelayAvailable;
	}

	public boolean isDepartureDelayAvailable() {
		return departureDelayAvailable;
	}

	public boolean isDistanceAvailable() {
		return distanceAvailable;
	}

	public String getUniqueCarrier() {
		return uniqueCarrier;
	}
	
	
	
}


com.adacho.mapper 패키지생성

DepartureDelayCountMapper 클래스생성

In [ ]:
package com.adacho.mapper;

import java.io.IOException;

import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.LongWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Mapper;

import com.adacho.common.AirlinePerformanceParser;

public class DepartureDelayCountMapper extends Mapper<LongWritable, Text, Text, IntWritable>{
	private final static IntWritable outputValue = new IntWritable(1);
	private Text outputKey = new Text();
	@Override
	protected void map(LongWritable key, Text value, Mapper<LongWritable, Text, Text, IntWritable>.Context context)
			throws IOException, InterruptedException {
		// TODO Auto-generated method stub
		AirlinePerformanceParser parser = new AirlinePerformanceParser(value);
		
		outputKey.set(parser.getYear() + "," + parser.getMonth()); //파서에 년도 콤마 월 붙여서 키를 만듦
		
		if(parser.getDepartureDelayTime() > 0) { //양수일때만
			context.write(outputKey, outputValue); // 딜레이와 1을 연결해서 context에 담음
		}
		
	}
	
	
}


com.adacho.reducer 패키지 생성

DelayCountReducer 클래스 생성

In [ ]:
package com.adacho.reducer;

import java.io.IOException;

import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Reducer;

public class DelayCountReducer extends Reducer<Text, IntWritable, Text, IntWritable>{
	private IntWritable result = new IntWritable();

	@Override
	protected void reduce(Text key, Iterable<IntWritable> values,
			Reducer<Text, IntWritable, Text, IntWritable>.Context context) throws IOException, InterruptedException {
		// TODO Auto-generated method stub
		int sum = 0;
	    for(IntWritable value: values) {
	       sum += value.get();
	    }
	    result.set(sum);
	    context.write(key, result);
	}

	
}


com.adacho.driver 패키지생성

DepartureDelayCount 클래스 생성

In [ ]:
package com.adacho.driver;

import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.input.TextInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;
import org.apache.hadoop.mapreduce.lib.output.TextOutputFormat;

import com.adacho.mapper.DepartureDelayCountMapper;
import com.adacho.reducer.DelayCountReducer;

public class DepartureDelayCount {

	public static void main(String[] args) throws Exception{
		Configuration conf = new Configuration();
		if(args.length !=2) {
			System.err.println("Usage: DepartureDelayCount <input> <output>");
			System.exit(1);
		}
		
		Job job = Job.getInstance(conf, "DepartureDelayCount");
		
		job.setJarByClass(DepartureDelayCount.class); //드라이브 클래스 저장
		job.setMapperClass(DepartureDelayCountMapper.class); //메퍼클래스저장
		job.setReducerClass(DelayCountReducer.class); //리듀스클래스저장
		
		job.setInputFormatClass(TextInputFormat.class); //두개중에 길이가 긴 것 임포트
		job.setOutputFormatClass(TextOutputFormat.class); //두개중에 길이가 긴 것 임포트
		
		job.setMapOutputKeyClass(Text.class);
		job.setMapOutputValueClass(IntWritable.class);
		
		FileInputFormat.addInputPath(job, new Path(args[0])); //여러개에 인풋데이타가 있으면 다 줄 수 있음
		FileOutputFormat.setOutputPath(job, new Path(args[1]));
		
		job.waitForCompletion(true); //job이끝날때까지 기다리고있다가 동기방식으로 
		
	}
}


Maven Clean , Maven install

c아래 javaworkspace 아래 airPerformance 아래 target 아래 jar파일생성확인 및 경로복사

윈도우 터미널에서 cd C:\java_workspace\airPerformance\target (경로)

ls로 airPerformance-0.1.jar 확인

scp .\airPerformance-0.1.jar ubuntu:~/work/java

윈도우에서 리눅스로 보내기.

start-dfs.sh

start-yarn.sh

jps

mapred historyserver start&

exit

cd work

cd java

yarn jar ./airPerformance-0.1.jar com.adacho.driver.DepartureDelayCount air-input dep-delay-count

hdfs dfs -ls dep-delay-count 로 확인

hdfs dfs -cat dep-delay-count/part-r-00000

만약 우리가올린파일이 128MB넘으면 00001 ~ 해서 파티션이 생김